In [2]:
# conda install pytorch torchvision torchaudio pytorch-cuda=12.2 -c pytorch -c nvidia

^C

Note: you may need to restart the kernel to use updated packages.


In [5]:
conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

Retrieving notices: ...working... done
Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: ...working... unsuccessful attempt using repodata from current_repodata.json, retrying with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Shruti\Software\anaconda

  added / updated specs:
    - pytorch
    - pytorch-cuda=12.1
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cuda-cccl-12.6.77          |                0          16 KB  nvidia
    cuda-libraries-dev-12.1.0  |                0           1 KB  nvidia
    cuda-nvrtc-12.1.105        |                0        73.2 MB  nvidia
    cuda-nvrtc-dev-12.1.105    |                0        16.5 MB  nvidia
    cuda-nvtx-12.1.105         |                0       



==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.11.0




Note: you may need to restart the kernel to use updated packages.

Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: ...working... unsuccessful attempt using repodata from current_repodata.json, retrying with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Shruti\Software\anaconda

  added / updated specs:
    - pytorch
    - pytorch-cuda=12.1
    - torchaudio
    - torchvision


The following NEW packages will be INSTALLED:

  cuda-cccl          nvidia/win-64::cuda-cccl-12.6.77-0 
  cuda-cccl_win-64   nvidia/noarch::cuda-cccl_win-64-12.6.77-0 
  cuda-cudart        nvidia/win-64::cuda-cudart-12.1.105-0 
  cuda-cudart-dev    nvidia/win-64::cuda-cudart-dev-12.1.105-0 
  cuda-cupti         nvidia/win-64::cuda-cupti-12.1.105-0 
  cuda-libraries     nvidia/win-64::cuda-libraries-12.1.0-0 
  cuda-libraries-dev nvidia/win-64::cuda-libraries-d



==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.11.0




In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

In [5]:
data_path = os.path.join("../Data")
train_data_path = os.path.join(data_path, "asl_alphabet_train/asl_alphabet_train")
test_data_path = os.path.join(data_path, "asl_alphabet_test/asl_alphabet_test")

In [6]:
train_transforms = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])

In [7]:
train_dataset = torchvision.datasets.ImageFolder(root=train_data_path, transform=train_transforms)

In [8]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=False)

In [9]:
def get_mean_std(loader) :
    mean = 0.
    std = 0.
    nb_samples = 0
    for data, _ in loader:
        batch_samples = data.size(0)
        data = data.view(batch_samples, data.size(1), -1)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
        nb_samples += batch_samples
        
    mean /= nb_samples
    std /= nb_samples
    
    return mean, std

In [10]:
mean, std = get_mean_std(train_loader)
mean
std

KeyboardInterrupt: 

In [ ]:
mean

In [ ]:
std

In [ ]:
mean = [0.5188, 0.4989, 0.5142]
std = [0.2023, 0.2314, 0.2404]
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

test_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

In [ ]:
class UnlabeledImageDataset(Dataset) :
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_paths = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self,idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(root = train_data_path, transform = train_transforms)
test_dataset = UnlabeledImageDataset(root_dir = test_data_path, transform = test_transforms)

In [ ]:
def show_transformed_images(dataset) :
    loader = torch.utils.data.DataLoader(dataset, batch_size=32,shuffle=True)
    batch = next(iter(loader))
    images, labels = batch
    
    grid = torchvision.utils.make_grid(images, nrow = 3)
    plt.figure(figsize=(11,11))
    plt.imshow(np.transpose(grid, (1,2,0)))
    print('labels : ', labels)

In [ ]:
show_transformed_images(train_dataset)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
torch.cuda.is_available()
# torch.cuda.device_count()

In [ ]:
def set_device():
    if torch.cuda.is_available():
        dev = "cuda:0"
    else:
        dev = "cpu"
    return torch.device(dev)

In [ ]:
def train_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs) :
    device = set_device()
    best_acc = 0
    
    for epoch in range(n_epochs) :
        print(f"epoch = {epoch + 1}")
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        
        for data in train_loader :
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            
            optimizer.zero_grad()
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels == predicted).sum().item()
            
        epoch_loss = running_loss/len(train_loader)
        epoch_acc = 100.00 * running_correct / total
        
        print(f"Train --- Running correct - {running_correct}, Total - {total}, Epochs - {epoch_acc}, Epoch Loss - {epoch_loss}")
        
        test_dataset_acc = eval_model_on_test(model, test_loader)
        
        if (test_dataset_acc > best_acc) :
            best_acc = test_dataset_acc
            save_checkpoint(model, epoch, optimizer, best_acc)
            
    print("Done!!")

    return model

In [ ]:
def save_checkpoint(model, epoch, optimizer, best_acc) :
    state = {
        'epoch' : epoch + 1,
        'model' : model.state_dict(),
        'best accuracy' : best_acc,
        'optimizer' : optimizer.state_dict(),
    }
    
    torch.save(state, 'model_best_checkpoint.pth.tar')

In [ ]:
def eval_model_on_test(model, test_loader):
    model.eval()
    predicted_correctly_on_epoch = 0
    total = 0
    device = set_device()
    
    with torch.no_grad() :
        for data in test_loader :
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            predicted_correctly_on_epoch += (predicted == labels).sum().item()
    
    epoch_acc = 100.00 * predicted_correctly_on_epoch / total

    print(f" Test --- Correct - {predicted_correctly_on_epoch} Total - {total} Epoch - {epoch_acc}")
    
    return epoch_acc

In [ ]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

resnet18_model = models.resnet18(pretrained=False)
num_ftrs = resnet18_model.fc.in_features
number_of_class = 29
resnet18_model.fc = nn.Linear(num_ftrs, number_of_class)
device = set_device()
resnet_18_model = resnet18_model.to(device)
loss_fn = nn.CrossEntropyLoss()

optimizer = optim.SGD(resnet18_model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.003)

In [ ]:
train_nn(resnet18_model, train_loader, test_loader, loss_fn, optimizer, 150)

In [ ]:
checkpoint = torch.load('model_best_checkpoint.pth.tar')

In [ ]:
resnet18_model = models.resnet18()
num_ftrs = resnet18_model.fc.in_features
number_of_class = 29
resnet18_model.fc = nn.Linear(num_ftrs, number_of_class)
resnet18_model.load_state_dict(checkpoint['model'])

torch.save(resnet18_model, 'best_model.pth')